In [1]:
"""Plot MED OFF vs. ON ImCoh (Figure 3a & 3b)."""

import os
import sys
from pathlib import Path
import numpy as np
import copy
import pandas as pd
import pte_stats
import pte_decode
from matplotlib import pyplot as plt
from scipy.stats import sem

cd_path = Path(os.getcwd()).absolute().parent
sys.path.append(os.path.join(cd_path, "coherence"))

import matplotlib
matplotlib.rc('xtick', labelsize=6)
matplotlib.rc('ytick', labelsize=6)
matplotlib.rc('legend', fontsize=6)
matplotlib.rc("font", size=6, family="Arial")
matplotlib.rc('axes', labelsize=7)
matplotlib.rc('axes', titlesize=7)
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42

prop_cycle = plt.rcParams["axes.prop_cycle"]

n_perm = 100_000
two_sided = True
alpha = 0.05

FOLDERPATH_ANALYSIS = "Path_to\\Project\\Analysis"
FOLDERPATH_FIGURES = os.path.join(os.path.dirname(os.getcwd()), "figures")

lfreq = 4.0
hfreq = 40.0

fband_func = np.mean

fbands = {
    "alpha": (8, 12),
    "low_beta": (12, 20),
    "high_beta": (20, 30)
}


In [ ]:
# WHOLE CORTEX <-> STN ImCoh (Figure 3b)
data = pd.read_pickle(os.path.join(
    FOLDERPATH_ANALYSIS, "task-Rest_acq-multi_run-multi_con_imcoh_whole-MedOffOn_multi_sub.pkl")
)
freqs = copy.deepcopy(data["frequencies"][0])
freqs_delta = np.unique(np.subtract(freqs[1:], freqs[:-1]))[0]
data["frequencies"] = [freqs] * len(data["seed_types"])
data = pd.DataFrame.from_dict(data)

imcoh_off = np.array(data["connectivity-imcoh"][data["med"] == "Off"].to_list())
imcoh_on = np.array(data["connectivity-imcoh"][data["med"] == "On"].to_list())

np.random.seed(44)

lfreq_i = freqs.index(lfreq)
hfreq_i = freqs.index(hfreq)
plot_freqs = freqs[lfreq_i:hfreq_i+1]

imcoh_off = imcoh_off[:, lfreq_i:hfreq_i+1]
imcoh_on = imcoh_on[:, lfreq_i:hfreq_i+1]

imcoh_off_sem = sem(imcoh_off)
imcoh_on_sem = sem(imcoh_on)

fig, axis = pte_decode.lineplot_prediction_compare(
    x_1=imcoh_off.T,
    x_2=imcoh_on.T,
    times=np.arange(lfreq, hfreq+0.5, 0.5),
    data_labels=["OFF therapy", "ON levodopa"],
    x_label="Frequency (Hz)",
    y_label="Absolute imaginary coherency (A.U.)",
    two_tailed=two_sided,
    paired_x1x2=True,
    n_perm=n_perm,
    correction_method="cluster_pvals",
    title="Cortex <--> STN",
    colour=["#DF4A4A", "#376EB4"]
)

for fband_name, fband_lims in fbands.items():
    stat, pval = pte_stats.permutation_onesample(
        data_a=fband_func(imcoh_off[:, freqs.index(fband_lims[0]):freqs.index(fband_lims[1])+1], 1),
        data_b=fband_func(imcoh_on[:, freqs.index(fband_lims[0]):freqs.index(fband_lims[1])+1], 1),
        n_perm=n_perm, two_tailed=two_sided
    )
    if pval < alpha:
        axis.axvspan(fband_lims[0], fband_lims[1], color=[0.8, 0.8, 0.8], alpha=0.3, linewidth=0)
        prefix = "[SIGNIFICANT]"
    else:
        prefix = ""
    print(f"{prefix} {fband_name}: stat={stat}; p-val={pval}")

axis.spines['top'].set_visible(False)
axis.spines['right'].set_visible(False)
axis.set_box_aspect(0.8)
axis.legend(handlelength=0.5, handletextpad=0.5, labelspacing=0, borderaxespad=0, loc="upper left")
leg = axis.get_legend()
leg.legend_handles[0].set_linewidth(1.5)
leg.legend_handles[1].set_linewidth(1.5)
leg.legend_handles[2].set_color([0.5, 0.5, 0.5])
leg.set_frame_on(False)
axis.set_xticks(np.arange(5, 45, 5))
axis.tick_params("x", pad=2, size=2)
axis.tick_params("y", pad=2, size=2)
axis.xaxis.labelpad = 0
axis.yaxis.labelpad = 1

fig.set_size_inches(2, 2)

fig.savefig(os.path.join(FOLDERPATH_FIGURES, "Manuscript_ImCoh_Med_whole_cortex-STN.pdf"))

In [ ]:
# WHOLE CORTEX <-> STN MIC (Figure 3a inset)
data = pd.read_pickle(os.path.join(
    FOLDERPATH_ANALYSIS, "task-Rest_acq-multi_run-multi_con_mim_mic_whole-MedOffOn_multi_sub.pkl")
)
freqs = copy.deepcopy(data["frequencies"][0])
freqs_delta = np.unique(np.subtract(freqs[1:], freqs[:-1]))[0]
data["frequencies"] = [freqs] * len(data["seed_types"])
data = pd.DataFrame.from_dict(data)

mic_off = np.array(data["connectivity-mic"][data["med"] == "Off"].to_list())
mic_on = np.array(data["connectivity-mic"][data["med"] == "On"].to_list())

np.random.seed(44)

lfreq = 5
hfreq = 35

lfreq_i = freqs.index(lfreq)
hfreq_i = freqs.index(hfreq)
plot_freqs = freqs[lfreq_i:hfreq_i+1]

mic_off = mic_off[:, lfreq_i:hfreq_i+1]
mic_on = mic_on[:, lfreq_i:hfreq_i+1]

mic_off_sem = sem(mic_off)
mic_on_sem = sem(mic_on)

fig, axis = plt.subplots(1, 1)

axis.plot(plot_freqs, mic_off.mean(0), color="#DF4A4A", label="OFF therapy")
axis.fill_between(plot_freqs, mic_off.mean(0)-mic_off_sem, mic_off.mean(0)+mic_off_sem, color="#DF4A4A", alpha=0.3)
axis.plot(plot_freqs, mic_on.mean(0), color="#376EB4", label="ON levodopa")
axis.fill_between(plot_freqs, mic_on.mean(0)-mic_on_sem, mic_on.mean(0)+mic_on_sem, color="#376EB4", alpha=0.3)

xlim = axis.get_xlim()
axis.axvspan(4, 20, color="white", alpha=0.6, edgecolor="none", linewidth=0, zorder=99)
axis.axvspan(30, 40, color="white", alpha=0.6, edgecolor="none", linewidth=0, zorder=99)
axis.set_xlim(xlim)

axis.spines['top'].set_visible(False)
axis.spines['right'].set_visible(False)
axis.set_xticks(np.arange(10, 36, 10))
axis.tick_params("x", pad=2, size=2)
axis.tick_params("y", pad=2, size=2)
axis.set_xlabel("Frequency (Hz)")
axis.set_ylabel("Abs. imag.\ncoherency (A.U.)")
fig.suptitle("Cortex <--> STN")
axis.xaxis.labelpad = 0.1
axis.yaxis.labelpad = 0

fig.set_size_inches(1, 1)

fig.savefig(os.path.join(FOLDERPATH_FIGURES, "Manuscript_MIC_Med_full_hbhighlight_whole_cortex-STN.pdf"))